In [1]:
import pandas as pd

# 1. Load the datasets
center_tract_data = pd.read_csv('PLACES__Local_Data_for_Better_Health__Census_Tract_Data_2024_release.csv')
zip_tract_crosswalk = pd.read_excel('HUD 2020 Zip Code - Census Track Crosswalk.xlsx')
life_expectancy = pd.read_csv('US_A.csv')

In [3]:
measure_category = {
    'Current lack of health insurance among adults aged 18-64 years' : 'Healthcare Access and Quality',
    'Visits to doctor for routine checkup within the past year among adults' : 'Healthcare Access and Quality',
    'Mammography use among women aged 50-74 years' : 'Healthcare Access and Quality',
    'Colorectal cancer screening among adults aged 45–75 years' : 'Healthcare Access and Quality',
    'Visited dentist or dental clinic in the past year among adults' : 'Healthcare Access and Quality',
    'Cholesterol screening among adults' : 'Healthcare Access and Quality',
    'Taking medicine to control high blood pressure among adults with high blood pressure' : 'Healthcare Access and Quality',
    'Received food stamps in the past 12 months among adults' : 'Food Quality',
    'Food insecurity in the past 12 months among adults' : 'Food Quality',
    # 'Housing insecurity in the past 12 months among adults' : 'Economic Stability',
    # 'Utility services shut-off threat in the past 12 months among adults' : 'Economic Stability',
    'Lack of reliable transportation in the past 12 months among adults' : 'Transportation',
    'Feeling socially isolated among adults' : 'socioeconomic',
    'Lack of social and emotional support among adults' : 'socioeconomic',
    'Fair or poor self-rated health status among adults' : 'Education Access and Health Literacy',
    'Frequent mental distress among adults' : 'Education Access and Health Literacy',
    'Cognitive disability among adults' : 'Education Access and Health Literacy',
    'Depression among adults' : 'Education Access and Health Literacy'
    # 'Current cigarette smoking among adults' : 'Neighborhood and Built Environment / Health Behaviors',
    # 'Binge drinking among adults' : 'Neighborhood and Built Environment / Health Behaviors',
    # 'No leisure-time physical activity among adults' : 'Neighborhood and Built Environment / Health Behaviors',
    # 'Obesity among adults' : 'Neighborhood and Built Environment / Health Behaviors',
    # 'Short sleep duration among adults' : 'Neighborhood and Built Environment / Health Behaviors'
}

In [4]:
center_tract_data = center_tract_data[center_tract_data['Measure'].isin(measure_category.keys())]
center_tract_data['Category'] = center_tract_data['Measure'].map(measure_category)

In [5]:
df = center_tract_data.pivot_table(
    index = 'LocationID',
    columns = 'Measure',
    values = 'Data_Value',
    aggfunc = 'mean'
).reset_index()

from collections import defaultdict
cat_measures = defaultdict(list)
for measure, cat in measure_category.items():
    cat_measures[cat].append(measure)

for cat,measures in cat_measures.items():
    df[cat] = df[measures].mean(axis=1)

In [6]:
file2_deduplicated = zip_tract_crosswalk.drop_duplicates(subset='TRACT')
# Merge both files on 'location_id'
merged_df = pd.merge(df, file2_deduplicated,left_on='LocationID', right_on='TRACT', how='left')

In [7]:
merged_df.columns

Index(['LocationID', 'Cholesterol screening among adults',
       'Cognitive disability among adults',
       'Colorectal cancer screening among adults aged 45–75 years',
       'Current lack of health insurance among adults aged 18-64 years',
       'Depression among adults',
       'Fair or poor self-rated health status among adults',
       'Feeling socially isolated among adults',
       'Food insecurity in the past 12 months among adults',
       'Frequent mental distress among adults',
       'Lack of reliable transportation in the past 12 months among adults',
       'Lack of social and emotional support among adults',
       'Mammography use among women aged 50-74 years',
       'Received food stamps in the past 12 months among adults',
       'Taking medicine to control high blood pressure among adults with high blood pressure',
       'Visited dentist or dental clinic in the past year among adults',
       'Visits to doctor for routine checkup within the past year among adult

In [9]:
columns_1 = [ 'Cholesterol screening among adults',
       'Cognitive disability among adults',
       'Colorectal cancer screening among adults aged 45–75 years',
       'Current lack of health insurance among adults aged 18-64 years',
       'Depression among adults',
       'Fair or poor self-rated health status among adults',
       'Feeling socially isolated among adults',
       'Food insecurity in the past 12 months among adults',
       'Frequent mental distress among adults',
       'Lack of reliable transportation in the past 12 months among adults',
       'Lack of social and emotional support among adults',
       'Mammography use among women aged 50-74 years',
       'Received food stamps in the past 12 months among adults',
       'Taking medicine to control high blood pressure among adults with high blood pressure',
       'Visited dentist or dental clinic in the past year among adults',
       'Visits to doctor for routine checkup within the past year among adults','ZIP', 'TRACT',
       'USPS_ZIP_PREF_CITY', 'USPS_ZIP_PREF_STATE', 'RES_RATIO', 'BUS_RATIO',
       'OTH_RATIO', 'TOT_RATIO']

DF = merged_df.copy()
DF = DF.drop(columns = columns_1,axis=1)

In [10]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scorecard_scaled = DF.copy()
scorecard_scaled[['Healthcare Access and Quality', 'Food Quality','Transportation', 'socioeconomic','Education Access and Health Literacy']] = scaler.fit_transform(
    DF[['Healthcare Access and Quality', 'Food Quality','Transportation', 'socioeconomic','Education Access and Health Literacy']]
)
X = scorecard_scaled.drop(columns=['LocationID'])

In [13]:
X.isnull().sum()

Healthcare Access and Quality           0
Food Quality                            0
Transportation                          0
socioeconomic                           0
Education Access and Health Literacy    0
dtype: int64

In [12]:
X = X.fillna(X.median())

In [14]:
thresholds = X.quantile(0.66)

def flag_issues(row):
    flags = {}
    for col in X.columns:
        flags[col] = '⚠️' if row[col] < thresholds[col] else '✅'
    return pd.Series(flags)

scorecard_scaled[[
    'Healthcare_Flag',
    'Food_Flag',
    'Transportaion_Flag',
    'Socioeconomic_Flag',
    'Education_Flag'
]] = X.apply(flag_issues, axis=1)

In [15]:
df_2 = scorecard_scaled.drop(columns = ['Healthcare Access and Quality','Food Quality','Transportation','socioeconomic','Education Access and Health Literacy'],axis=1)
merged_scorecard = pd.merge(df_2, file2_deduplicated,left_on='LocationID', right_on='TRACT', how='left')
merged_scorecard = merged_scorecard.drop(columns = ['TRACT','USPS_ZIP_PREF_CITY','USPS_ZIP_PREF_STATE'],axis=1)

In [16]:
flag_map = {'⚠️': 1, '✅': 0}

sdh_flags = [
    'Healthcare_Flag',
    'Food_Flag',
    'Transportaion_Flag',
    'Socioeconomic_Flag',
    'Education_Flag'
]

for col in sdh_flags:
    merged_scorecard[col] = merged_scorecard[col].map(flag_map)

In [17]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier

features = ['RES_RATIO', 'BUS_RATIO', 'OTH_RATIO', 'TOT_RATIO','ZIP']  # ZIP-level input columns
X = merged_scorecard[features]
y = merged_scorecard[sdh_flags]

In [18]:
from sklearn.ensemble import RandomForestClassifier

models = {}

for col in sdh_flags:
    target_flag = f'{col}'
    model = RandomForestClassifier(random_state=42)
    model.fit(merged_scorecard[features], merged_scorecard[target_flag])
    models[target_flag] = model

In [19]:
def predict_all_flags(zip_input_dict):
    import numpy as np
    X_input = np.array([zip_input_dict[feat] for feat in features]).reshape(1, -1)
    
    predictions = {}
    for flag, model in models.items():
        predictions[flag] = model.predict(X_input)[0]
    
    return predictions

In [20]:
from sklearn.metrics import accuracy_score

for flag in sdh_flags:
    y_true = merged_scorecard[flag]
    y_pred = models[flag].predict(merged_scorecard[features])
    acc = accuracy_score(y_true, y_pred)
    print(f"{flag} Accuracy: {acc:.3f}")

Healthcare_Flag Accuracy: 0.777
Food_Flag Accuracy: 0.802
Transportaion_Flag Accuracy: 0.802
Socioeconomic_Flag Accuracy: 0.827
Education_Flag Accuracy: 0.771


In [21]:
DF_22 = merged_df.copy()
DF_22 = DF_22.drop(columns=['Healthcare Access and Quality', 'Food Quality','Transportation', 'socioeconomic','Education Access and Health Literacy','ZIP', 'TRACT',
       'USPS_ZIP_PREF_CITY', 'USPS_ZIP_PREF_STATE', 'RES_RATIO', 'BUS_RATIO',
       'OTH_RATIO', 'TOT_RATIO'],axis=1)
# Drop the location column to compute thresholds only on data columns
data_cols = DF_22.columns.drop('LocationID')
thresholds = DF_22[data_cols].quantile(0.66)

# Apply flags: 1 if value < threshold else 0
for col in data_cols:
    DF_22[col + '_Flag'] = (DF_22[col] < thresholds[col]).astype(int)


In [22]:
columns_22 = ['Cholesterol screening among adults',
       'Cognitive disability among adults',
       'Colorectal cancer screening among adults aged 45–75 years',
       'Current lack of health insurance among adults aged 18-64 years',
       'Depression among adults',
       'Fair or poor self-rated health status among adults',
       'Feeling socially isolated among adults',
       'Food insecurity in the past 12 months among adults',
       'Frequent mental distress among adults',
       'Lack of reliable transportation in the past 12 months among adults',
       'Lack of social and emotional support among adults',
       'Mammography use among women aged 50-74 years',
       'Received food stamps in the past 12 months among adults',
       'Taking medicine to control high blood pressure among adults with high blood pressure',
       'Visited dentist or dental clinic in the past year among adults',
       'Visits to doctor for routine checkup within the past year among adults'] 
DF_22 = DF_22.drop(columns = columns_22,axis=1)

In [24]:
# Map model-flag names to the human-readable category names used in measure_category
flag_to_category = {
    'Healthcare_Flag'   : 'Healthcare Access and Quality',
    'Food_Flag'     : 'Food Quality',
    'Transportaion_Flag' : 'Transportation',
    'Socioeconomic_Flag': 'socioeconomic',
    'Education_Flag'    : 'Education Access and Health Literacy'
}

def generate_scorecard(location_id: int, input_features: dict):
    """
    1. Predict the 5 main category flags for the new data point.
    2. Pull that location’s row from DF_22.
    3. For every main category predicted = 1, return the individual measure flags.
    """
    # 1. predict 1/0 for the 5 main categories
    main_preds = predict_all_flags(input_features)          # {'Healthcare_Flag': 1, ...}

    # 2. row for this location (assumes DF_22 already loaded)
    row = DF_22.loc[DF_22['LocationID'] == location_id].iloc[0]

    # 3. build detailed scorecard
    scorecard = {}
    for flag, pred in main_preds.items():
        if pred == 1:                                       # only if category is flagged
            category = flag_to_category[flag]
            scorecard[category] = {
                measure: int(row[measure + '_Flag'])                  # 1/0 from DF_22
                for measure, cat_name in measure_category.items()
                if cat_name == category
            }
    return scorecard



In [25]:
# -------- Example ----------
# location_id you want to review
loc_id = 1001020100  

# dict with values for every feature in your `features` list
input_dict = {
    'RES_RATIO': 0.3,
    'BUS_RATIO': 0.1,
    'OTH_RATIO': 0.1,
    'TOT_RATIO': 0.27,
    'ZIP': 36067
}

scorecard = generate_scorecard(loc_id, input_dict)
print(scorecard)


{'Healthcare Access and Quality': {'Current lack of health insurance among adults aged 18-64 years': 1, 'Visits to doctor for routine checkup within the past year among adults': 1, 'Mammography use among women aged 50-74 years': 0, 'Colorectal cancer screening among adults aged 45–75 years': 1, 'Visited dentist or dental clinic in the past year among adults': 1, 'Cholesterol screening among adults': 1, 'Taking medicine to control high blood pressure among adults with high blood pressure': 0}, 'Food Quality': {'Received food stamps in the past 12 months among adults': 1, 'Food insecurity in the past 12 months among adults': 1}, 'Transportation': {'Lack of reliable transportation in the past 12 months among adults': 1}, 'socioeconomic': {'Feeling socially isolated among adults': 0, 'Lack of social and emotional support among adults': 1}}


C:\Users\DELL\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
C:\Users\DELL\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
C:\Users\DELL\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
C:\Users\DELL\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
C:\Users\DELL\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feat

In [26]:
import pandas as pd
from IPython.display import display, HTML

def generate_summary_table(scorecard):
    rows = []

    for category, indicators in scorecard.items():
        triggered_indicators = [
            indicator for indicator, flag in indicators.items() if flag == 1
        ]
        if triggered_indicators:
            rows.append({
                'Determinants': category,
                'Example Indicators': "<ul>" + "".join(f"<li>{i}</li>" for i in triggered_indicators) + "</ul>"
            })

    df_summary = pd.DataFrame(rows)
    return df_summary

def display_summary_html(df_summary):
    styled_html = df_summary.to_html(escape=False, index=False)
    display(HTML(styled_html))


In [27]:
summary_df = generate_summary_table(scorecard)
display_summary_html(summary_df)

Determinants,Example Indicators
Healthcare Access and Quality,Current lack of health insurance among adults aged 18-64 yearsVisits to doctor for routine checkup within the past year among adultsColorectal cancer screening among adults aged 45–75 yearsVisited dentist or dental clinic in the past year among adultsCholesterol screening among adults
Food Quality,Received food stamps in the past 12 months among adultsFood insecurity in the past 12 months among adults
Transportation,Lack of reliable transportation in the past 12 months among adults
socioeconomic,Lack of social and emotional support among adults


## solution

In [28]:
hospital_df = pd.read_excel('Copy of Hospital_General_Information.xlsx')
homehealth_df = pd.read_excel('Copy of HH_Provider_Jan2024.xlsx')
supplier_df = pd.read_excel('Copy of Medical-Equipment-Suppliers.xlsx')

In [29]:
def get_healthcare_solution(zipcode: int) -> str:
    """
    Returns the hospital or home health provider name and phone based on ZIP code.
    Priority: Hospital → Home Health. Returns fallback message if not found.
    """
    # Search in hospital dataset
    match = hospital_df[hospital_df['ZIP Code'] == zipcode]
    if not match.empty:
        name  = match.iloc[0]['Facility Name']
        phone = match.iloc[0]['Telephone Number']
        return f"{name} – {phone}"

    # If not found, search in home health dataset
    match = homehealth_df[homehealth_df['zipcode'] == zipcode]
    if not match.empty:
        name  = match.iloc[0]['Provider Name']
        phone = match.iloc[0]['Telephone Number']
        return f"{name} – {phone}"

    # If neither found
    return "No provider found"


In [30]:
zip_input = 36067
solution = get_healthcare_solution(zip_input)
print(solution)

PRATTVILLE BAPTIST HOSPITAL – (334) 361-4267


In [31]:
merged_zip_place_df = pd.merge(center_tract_data, file2_deduplicated,left_on='LocationID', right_on='TRACT', how='left')
import numpy as np
import math

def _haversine(lon1, lat1, lon2, lat2):
    """Great-circle distance in kilometers."""
    R = 6371.0  # Earth radius (km)
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])
    dlon  = lon2 - lon1
    dlat  = lat2 - lat1
    a = np.sin(dlat/2)**2 + np.cos(lat1)*np.cos(lat2)*np.sin(dlon/2)**2
    return 2 * R * np.arcsin(np.sqrt(a))

def get_nearest_supplier(zipcode: int):
    row = merged_zip_place_df .loc[merged_zip_place_df ['ZIP'] == zipcode]
    if row.empty:
        return None  # zip not in table
    point_str = row.iloc[0]['Geolocation']           # "POINT (-80.25 25.96)"
    lon0, lat0 = map(float, point_str.strip('POINT ()').split())
    
    # ---- 2. distances to suppliers
    lons = supplier_df['longitude'].to_numpy(dtype=float)
    lats = supplier_df['latitude' ].to_numpy(dtype=float)
    dists = _haversine(lon0, lat0, lons, lats)
    
    if len(dists) == 0:
        return None  # no suppliers data
    idx = int(np.argmin(dists))
    
    # ---- 3. build answer
    best = supplier_df.iloc[idx]
    return {
        'supplier_name' : best['practicename'],
        'Address'      : str(best['practiceaddress1']),
        'City'     : (best['practicecity']),
        'distance_km'   : float(dists[idx])
    }


In [32]:
result_3332 = get_nearest_supplier(33030)
print(result_3332)

{'supplier_name': 'PRICE CHOPPER PHARMACY #240', 'Address': '2 WORCESTER ROAD', 'City': 'WORCESTER', 'distance_km': nan}


In [34]:
supplier_df[['latitude', 'longitude']].isna().sum()
supplier_df = supplier_df.dropna(subset=['latitude', 'longitude'])

Now webscrapping is done and csv files are made with facility name, address and number

In [48]:
def get_trans(zipcode):
    testttt_1 = pd.read_csv("transportation_services_2.csv")
    match = testttt_1[testttt_1['search_zip'] == zipcode]
    if not match.empty:
        row = match.iloc[0]
        return f'name = "{row["name"]}"\naddress = "{row["address"]}"\nnumber = "{row["phone"]}"'


def get_socio(zipcode):
    testttt_2 = pd.read_csv("socioeconomic_2.csv")
    match = testttt_2[testttt_2['search_zip'] == zipcode]
    if not match.empty:
        row = match.iloc[0]
        return f'name = "{row["name"]}"\naddress = "{row["address"]}"\nnumber = "{row["phone"]}"'


def get_edu(zipcode):
    testttt_3 = pd.read_csv("Education.csv")
    match = testttt_3[testttt_3['search_zip'] == zipcode]
    if not match.empty:
        row = match.iloc[0]
        return f'name = "{row["name"]}"\naddress = "{row["address"]}"\nnumber = "{row["phone"]}"'


def get_food(zipcode):
    testttt_4 = pd.read_csv("food_centers_output.csv")
    match = testttt_4[testttt_4['zipcode'] == zipcode]
    if not match.empty:
        row = match.iloc[0]
        return f'name = "{row["name"]}"\naddress = "{row["address"]}"\nnumber = "{row["phone"]}"'

In [68]:
# dif_2 = pd.read_csv('socioeconomic_2.csv')
# print(dif_2.head(12))

In [69]:
# zipcode_input = 36067
# print(get_socio(zipcode_input))

In [76]:
import pandas as pd
from IPython.display import HTML, display
import inspect  # ✅ required for safe_call()


# ── 1.  MAP EACH DETERMINANT TO ITS SOLUTION FUNCTION ──────────────────────────
#    Replace A-E with your real functions.
solution_funcs = {
   "healthcare access and quality": [get_healthcare_solution, get_nearest_supplier],
    "food quality": [get_food],
    "transportation": [get_trans],
    "socioeconomic": [get_socio],
    "education access and health literacy": [get_edu],
}


In [77]:
def safe_call(func, triggered, zipcode):
    if len(inspect.signature(func).parameters) == 2:
        return func(triggered, zipcode)
    else:
        return func(zipcode)


# Updated generate_summary_table
def generate_summary_table(scorecard, zipcode, func_map=solution_funcs):
    rows = []

    for category, indicators in scorecard.items():
        triggered = [ind for ind, flag in indicators.items() if flag == 1]
        if not triggered:
            continue

        category_lower = category.lower()
        solutions = []
        for fn in func_map.get(category_lower, []):
            try:
                solutions.append(safe_call(fn, triggered, zipcode))
            except Exception as e:
                solutions.append(f"Error: {e}")

        rows.append({
            "Determinants": category,
            "Example Indicators": "<ul>" + "".join(f"<li>{i}</li>" for i in triggered) + "</ul>",
            "Solution": "<ul>" + "".join(f"<li>{s}</li>" for s in solutions) + "</ul>"
        })

    return pd.DataFrame(rows)

In [80]:
# Display HTML
def display_summary_html(df_summary):
    display(HTML(df_summary.to_html(escape=False, index=False)))
# ✅ Usage
# python
# Copy
# Edit
input_dict = {
    'RES_RATIO': 0.3,
    'BUS_RATIO': 0.1,
    'OTH_RATIO': 0.1,
    'TOT_RATIO': 0.27,
    'ZIP': 36067
}

scorecard = generate_scorecard(loc_id, input_dict)

C:\Users\DELL\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
C:\Users\DELL\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
C:\Users\DELL\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
C:\Users\DELL\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
C:\Users\DELL\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feat

In [81]:
summary_df = generate_summary_table(scorecard, input_dict['ZIP'])
display_summary_html(summary_df)

Determinants,Example Indicators,Solution
Healthcare Access and Quality,Current lack of health insurance among adults aged 18-64 yearsVisits to doctor for routine checkup within the past year among adultsColorectal cancer screening among adults aged 45–75 yearsVisited dentist or dental clinic in the past year among adultsCholesterol screening among adults,"PRATTVILLE BAPTIST HOSPITAL – (334) 361-4267{'supplier_name': 'PROHEALTH MEDICAL', 'Address': '213 S COURT ST', 'City': 'PRATTVILLE', 'distance_km': 1.635247218222527}"
Food Quality,Received food stamps in the past 12 months among adultsFood insecurity in the past 12 months among adults,"name = ""Montomery Area Council On Aging""\naddress = ""115 E Jefferson St, Montgomery, AL 36104-3616""\nnumber = ""(334)263-0532"""
Transportation,Lack of reliable transportation in the past 12 months among adults,"name = ""Yellow Cab Co Inc""\naddress = ""106 Tichnor Ave Prattville AL 36067-3036""\nnumber = ""(334) 361-6800"""
socioeconomic,Lack of social and emotional support among adults,"name = ""Unity Baptist Adult Day Care""\naddress = ""171 Benton Rd Lowndesboro AL 36752-4711""\nnumber = ""(334) 872-9901"""
